In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Project folder path
path = r'C:\Users\nanu_\Documents\Career Foundry\Projects\2 Data Immersion\Python fundamental for Data Analysis\01-2024 Instacart Basket analysis'


In [3]:
path

'C:\\Users\\nanu_\\Documents\\Career Foundry\\Projects\\2 Data Immersion\\Python fundamental for Data Analysis\\01-2024 Instacart Basket analysis'

In [4]:
# Importing ords_prods_merge Dataframe 
df_ords_prods_merge_final = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_final.pkl'))

In [5]:
# Checking the Dataframe
df_ords_prods_merge_final.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_day,days_since_prior_order,First_Order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days_modified,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,Not First Order,5,0,both,Mid-range product,Regularly busy,Regular days,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,Not First Order,1,1,both,Mid-range product,Regularly busy,Regular days,Most orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,Not First Order,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,First Order,10,0,both,Mid-range product,Regularly busy,Slowest days,Most orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,Not First Order,11,1,both,Mid-range product,Least busy,Slowest days,Most orders


In [6]:
# Calculation mean of the order_number by department_id from the data frame
df_ords_prods_merge_final.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


###  We see same number of departments in both sample and entire dataset but the value of mean is higher in some of the Departments in the sample dataset.

# Create Loyalty Flag for Existing Customers

In [7]:
# Use groupby and transform() functions with np.max argument to determine maximum order number for each user
df_ords_prods_merge_final['max_order'] =df_ords_prods_merge_final.groupby(['user_id'])['order_number'].transform(np.max) 

In [8]:
# Checking the Results
df_ords_prods_merge_final.loc[:,['user_id', 'order_number']].head()

,user_id,order_number
0,138,28
1,138,30
2,709,2
3,764,1
4,764,3


# Use loc function to make loyalty flag column with given criteria

In [9]:
# Use loc function to make loyalty flag column with given criteria
df_ords_prods_merge_final.loc[df_ords_prods_merge_final['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
df_ords_prods_merge_final.loc[(df_ords_prods_merge_final['max_order'] <= 40) & (df_ords_prods_merge_final['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [11]:
df_ords_prods_merge_final.loc[df_ords_prods_merge_final['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
# Counting the value max_order
df_ords_prods_merge_final['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [14]:
# Checking the result of updated dataframe
df_ords_prods_merge_final.loc[:,['user_id','max_order', 'loyalty_flag']].head(10)

,user_id,max_order,loyalty_flag
0,138,32,Regular customer
1,138,32,Regular customer
2,709,5,New customer
3,764,3,New customer
4,764,3,New customer
5,777,26,Regular customer
6,825,9,New customer
7,910,12,Regular customer
8,1052,20,Regular customer
9,1052,20,Regular customer


# Check the basic statistics of the product prices for each loyalty category

In [15]:
# performing basic statistics of spending habits by loyalty group
df_ords_prods_merge_final.groupby('loyalty_flag').agg({'prices': ['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

#### Loyal customer pays around 10 dollors which is 3 dollars less than new customer and 2 dollors less than Regular customers. which shows Loyal customer pays less than other customer

# Aggregating Data with transform()

In [16]:
# Use transform() to determine the mean price of products purchased by each user
df_ords_prods_merge_final['mean_product_price'] = df_ords_prods_merge_final.groupby(['user_id'])['prices'].transform(np.mean)

In [17]:
# Checking the result of Dataframe
df_ords_prods_merge_final.loc[:,['user_id','mean_product_price']].head(10)

,user_id,mean_product_price
0,138,6.935811
1,138,6.935811
2,709,7.930208
3,764,4.972414
4,764,4.972414
5,777,6.935398
6,825,5.957576
7,910,6.680000
8,1052,7.162500
9,1052,7.162500


## Creating Spending_flag using loc() function 

In [18]:
df_ords_prods_merge_final.loc[df_ords_prods_merge_final['mean_product_price'] < 10, 'spending_flag'] = 'Low spender'

In [19]:
df_ords_prods_merge_final.loc[df_ords_prods_merge_final['mean_product_price'] >= 10, 'spending_flag'] = 'High spender'


In [20]:
# Counting the value of spending_flag
df_ords_prods_merge_final['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

#### The output shows that there are more low spenders.

In [22]:
# Checking the result of spending_flag column
df_ords_prods_merge_final.loc[:,['user_id' , 'mean_product_price', 'spending_flag']].head(10)

,user_id,mean_product_price,spending_flag
0,138,6.935811,Low spender
1,138,6.935811,Low spender
2,709,7.930208,Low spender
3,764,4.972414,Low spender
4,764,4.972414,Low spender
5,777,6.935398,Low spender
6,825,5.957576,Low spender
7,910,6.680000,Low spender
8,1052,7.162500,Low spender
9,1052,7.162500,Low spender


In [23]:
# Use groupby and transform to determine median number of days between orders for each user
df_ords_prods_merge_final['median_days_between_orders'] = df_ords_prods_merge_final.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [24]:
# Checking the result of Dataframe
df_ords_prods_merge_final.loc[:,['user_id' , 'median_days_between_orders']].head(10)

,user_id,median_days_between_orders
0,138,8.0
1,138,8.0
2,709,8.0
3,764,9.0
4,764,9.0
5,777,11.0
6,825,20.0
7,910,6.0
8,1052,10.0
9,1052,10.0


## Creating order frequency using loc() function 

In [25]:
df_ords_prods_merge_final.loc[df_ords_prods_merge_final['median_days_between_orders'] > 20, 'order_frequency'] = 'Non-frequent customer'

In [26]:
df_ords_prods_merge_final.loc[(df_ords_prods_merge_final['median_days_between_orders'] <= 20) & (df_ords_prods_merge_final['median_days_between_orders'] > 10), 'order_frequency'] = 'Regular customer'

In [27]:
df_ords_prods_merge_final.loc[df_ords_prods_merge_final['median_days_between_orders'] <= 10, 'order_frequency'] = 'Frequent customer'

In [28]:
# Counting the value of order_frequency
df_ords_prods_merge_final['order_frequency'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: order_frequency, dtype: int64

#### Five users have not made second order. Most of the Customers are frequent customers.

In [29]:
# Checking the result of Dataframe
df_ords_prods_merge_final.loc[:,['user_id' , 'median_days_between_orders', 'order_frequency']].head(10)

,user_id,median_days_between_orders,order_frequency
0,138,8.0,Frequent customer
1,138,8.0,Frequent customer
2,709,8.0,Frequent customer
3,764,9.0,Frequent customer
4,764,9.0,Frequent customer
5,777,11.0,Regular customer
6,825,20.0,Regular customer
7,910,6.0,Frequent customer
8,1052,10.0,Frequent customer
9,1052,10.0,Frequent customer


In [30]:
# Export as pickle file 
# Export derived data frame as pickle file
df_ords_prods_merge_final.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_grouped.pkl'))